<center><h1>Luo_Xiao_HW3</h1></center>
<br>
<br>

Name: Xiao Luo
<br>
Github Username: xluo23
<br>
USC ID: 2392693288

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [34]:
import os
import glob
import pandas as pd

import numpy as np
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats

Get the AReM Data Set

In [2]:
#read the path
folder_path = "/Users/luoxiao/Downloads/USC/Courses/dsci552_23sp/HW3/AReM/"

#list all the files from the directory
df_all = pd.DataFrame()
for name in ["bending1", "bending2", "cycling", "lying", "sitting", "standing", "walking"]:
    file_path = folder_path + name
    file_list = os.listdir(file_path)
    csv_files = list(filter(lambda f: f.endswith('.csv'), file_list))
    
    #append all files together
    for file in csv_files:
        file_name = file_path + "/" + file

        df_temp = pd.read_csv(file_name, skiprows=[0,1,2,3], skip_blank_lines=True, usecols=range(7))
        df_temp = df_temp.rename(columns={"# Columns: time": "time"})
        if name == "bending2" and file == "dataset4.csv":
            df_temp = pd.DataFrame(df_temp.time.str.split(' ', 6).tolist(),
                         columns = ['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'])
        df_temp = df_temp.assign(activity = name)  
        df_all = pd.concat([df_all, df_temp])
        # print(df_all)

print(df_all)

       time avg_rss12 var_rss12 avg_rss13 var_rss13 avg_rss23 var_rss23  \
0         0      42.0       0.0      18.5       0.5      12.0       0.0   
1       250      42.0       0.0      18.0       0.0     11.33      0.94   
2       500     42.75      0.43     16.75      1.79     18.25      0.43   
3       750      42.5       0.5     16.75      0.83      19.0      1.22   
4      1000      43.0      0.82     16.25      0.83      18.0       0.0   
..      ...       ...       ...       ...       ...       ...       ...   
475  118750      31.5      1.66      12.5       3.2     14.25      4.44   
476  119000     27.33      1.25     11.33      0.94      20.0       4.0   
477  119250      37.8      7.68      14.2      2.48     17.25      0.83   
478  119500     33.75       1.3     15.75      5.21      16.5      2.69   
479  119750     32.67      3.09     18.67      0.47      14.0      3.16   

     activity  
0    bending1  
1    bending1  
2    bending1  
3    bending1  
4    bending1  
.. 

### (b) Test and Train Data

In [3]:
df_train = pd.DataFrame()
df_test = pd.DataFrame()

for name in ["bending1", "bending2", "cycling", "lying", "sitting", "standing", "walking"]:
    file_path = folder_path + name
    file_list = os.listdir(file_path)
    csv_files = list(filter(lambda f: f.endswith('.csv'), file_list))
    
    for file in csv_files:
        file_name = file_path + "/" + file

        df_temp = pd.read_csv(file_name, skiprows=[0,1,2,3], skip_blank_lines=True, usecols=range(7))
        df_temp = df_temp.rename(columns={"# Columns: time": "time"})
        if name == "bending2" and file == "dataset4.csv":
            df_temp = pd.DataFrame(df_temp.time.str.split(' ', 6).tolist(),
                         columns = ['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'])
        df_temp = df_temp.assign(activity = name)  
        
        if file == "dataset1.csv" or file == "dataset2.csv":
            df_test = pd.concat([df_test, df_temp])
        elif file == "dataset3.csv": 
            if name == "bending1" or name == "bending2":
                df_train = pd.concat([df_train, df_temp])
            else: df_test = pd.concat([df_test, df_temp])
        else: df_train = pd.concat([df_train, df_temp])

print("The test data:\n", df_test)
print("The train data:\n",df_train)

The test data:
        time  avg_rss12  var_rss12  avg_rss13  var_rss13  avg_rss23  var_rss23  \
0         0      39.25       0.43      22.75       0.43      33.75       1.30   
1       250      39.25       0.43      23.00       0.00      33.00       0.00   
2       500      39.25       0.43      23.25       0.43      33.00       0.00   
3       750      39.50       0.50      23.00       0.71      33.00       0.00   
4      1000      39.50       0.50      24.00       0.00      33.00       0.00   
..      ...        ...        ...        ...        ...        ...        ...   
475  118750      36.00       2.45      17.00       5.10      20.50       0.87   
476  119000      34.33       1.89      15.00       2.45      17.00       2.12   
477  119250      33.00       7.35      14.60       3.14      13.00       5.70   
478  119500      31.67       1.25      11.00       6.16      19.25       2.17   
479  119750      30.75      10.21      11.75       1.09      18.50       3.20   

     activi

### (c) Feature Extraction

#### i. Research

In [4]:
print("Minimum, maximum, mean, median, standard deviation, quantiles(first quartile, third quartile and interquartile ranges) are usually used in time series classification.")

Minimum, maximum, mean, median, standard deviation, quantiles(first quartile, third quartile and interquartile ranges) are usually used in time series classification.


#### ii. Extraction

In [35]:
time_series = ['avg_rss12','var_rss12','avg_rss13','var_rss13','avg_rss23','var_rss23']
features = ['Min', 'Max', 'Mean', 'Median', 'SD', 'Q1', 'Q3']
columns = []
for ts in time_series:
    for feature in features:
        name = feature + "_" + ts
        columns.append(name)
# print(columns)

df = pd.DataFrame(columns = columns)

for name in ["bending1", "bending2", "cycling", "lying", "sitting", "standing", "walking"]:
    file_path = folder_path + name
    file_list = os.listdir(file_path)
    csv_files = list(filter(lambda f: f.endswith('.csv'), file_list))

    for file in csv_files:
        file_name = file_path + "/" + file
        df_temp = pd.read_csv(file_name, skiprows=[0,1,2,3], skip_blank_lines=True, usecols=range(7))
        df_temp = df_temp.rename(columns={"# Columns: time": "time"})
        if name == "bending2" and file == "dataset4.csv":
            df_temp = pd.DataFrame(df_temp.time.str.split(' ', 6).tolist(),
                            columns = ['time', 'avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23'])
            df_temp = df_temp.astype(float)

        row = []
        for i in time_series:
            ts_min = df_temp[i].min()
            ts_max = df_temp[i].max()
            ts_mean = df_temp[i].mean()
            ts_median = df_temp[i].median()
            ts_sd = df_temp[i].std()
            ts_q1 = df_temp[i].quantile(0.25)
            ts_q3 = df_temp[i].quantile(0.75)
            row.append(ts_min)
            row.append(ts_max)
            row.append(ts_mean)
            row.append(ts_median)
            row.append(ts_sd)
            row.append(ts_q1)
            row.append(ts_q3)

        df.loc[len(df.index)] = row
        
# df.insert(loc = 0,
#           column = 'Instance',
#           value = list(range(1,89)))
display(df)

,Min_avg_rss12,Max_avg_rss12,Mean_avg_rss12,Median_avg_rss12,SD_avg_rss12,Q1_avg_rss12,Q3_avg_rss12,Min_var_rss12,Max_var_rss12,Mean_var_rss12,...,SD_avg_rss23,Q1_avg_rss23,Q3_avg_rss23,Min_var_rss23,Max_var_rss23,Mean_var_rss23,Median_var_rss23,SD_var_rss23,Q1_var_rss23,Q3_var_rss23
0,36.25,48.00,43.969125,44.500,1.618364,43.31,44.67,0.0,1.50,0.413125,...,3.318301,20.5000,23.75,0.00,2.96,0.555313,0.490,0.487826,0.0000,0.8300
1,37.00,48.00,43.454958,43.250,1.386098,42.50,45.00,0.0,1.58,0.378083,...,2.488862,22.2500,24.00,0.00,5.26,0.679646,0.500,0.622534,0.4300,0.8700
2,33.00,47.75,42.179813,43.500,3.670666,39.15,45.00,0.0,3.00,0.696042,...,3.849448,30.4575,36.33,0.00,2.18,0.613521,0.500,0.524317,0.0000,1.0000
3,33.00,45.75,41.678063,41.750,2.243490,41.33,42.75,0.0,2.83,0.535979,...,2.411026,28.4575,31.25,0.00,1.79,0.383292,0.430,0.389164,0.0000,0.5000
4,37.25,45.00,40.624792,40.500,1.476967,39.25,42.00,0.0,1.30,0.358604,...,2.188449,33.0000,36.00,0.00,1.92,0.570583,0.430,0.582915,0.0000,1.3000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,19.75,45.50,34.322750,35.250,4.752477,31.00,38.00,0.0,13.47,4.456333,...,3.119856,13.5000,17.75,0.00,9.67,3.432562,3.200,1.732727,2.1575,4.5650
84,19.25,44.00,34.473188,35.000,4.796705,31.25,38.00,0.0,13.86,4.359312,...,3.156320,13.7300,17.75,0.43,9.00,3.340458,3.090,1.699114,2.1200,4.3750
85,23.50,46.25,34.873229,35.250,4.531720,31.75,38.25,0.0,14.82,4.380583,...,3.131076,13.7500,18.00,0.00,9.51,3.424646,3.270,1.690960,2.1700,4.5000
86,18.33,45.75,34.599875,35.125,4.731790,31.50,38.00,0.0,15.37,4.398833,...,2.905688,14.0000,18.25,0.00,8.86,3.289542,3.015,1.680170,2.1200,4.2600


#### iii. Standard Deviation

In [31]:
sd_estimate = pd.DataFrame(df.std())
sd_estimate.columns=['estimate_sd']
# display(sd_estimate)

CI = pd.DataFrame(columns=["90% CI for sd", "Lower Bound", "Upper Bound"])
for index, colname in enumerate(df):
    arr = np.asarray(df[colname]).astype(np.float64)
    feature_CI = bs.bootstrap(arr, stat_func = bs_stats.std, alpha=0.1)
    # print(feature_CI)
    CI.loc[colname, "90% CI for sd"] = feature_CI.value
    CI.loc[colname, "Lower Bound"] = feature_CI.lower_bound
    CI.loc[colname, "Upper Bound"] = feature_CI.upper_bound
    
Bootstrap_SD = pd.concat([sd_estimate, CI], ignore_index = True, axis = 1)
Bootstrap_SD.columns=['Estimate SD','90% Confidence Intervals for SD', "Lower Bound", "Upper Bound"]
display(Bootstrap_SD)

,Estimate SD,90% Confidence Intervals for SD,Lower Bound,Upper Bound
Min_avg_rss12,9.569975,9.515445,8.326561,10.792137
Max_avg_rss12,4.394362,4.369322,3.484021,5.415289
Mean_avg_rss12,5.335718,5.305314,4.774001,5.944455
Median_avg_rss12,5.440054,5.409056,4.837402,6.046564
SD_avg_rss12,1.772153,1.762056,1.585618,1.966892
Q1_avg_rss12,6.153590,6.118526,5.62716,6.700633
Q3_avg_rss12,5.138925,5.109643,4.415098,5.927576
Min_var_rss12,0.000000,0.0,0.0,0.0
Max_var_rss12,5.062729,5.033882,4.694647,5.473737
Mean_var_rss12,1.574164,1.565194,1.433112,1.742458


#### iv. Select Features

Three most important time-domain features:

1.Mean: help us know the central tendency of time series.

2.Median: in some case that outliers affect the mean value of the data, the median can help us know the central tendency that is less affected by outliers.

3.Standard Deviation: help us know how data spread out and how data vary from one another.

## 2. ISLR 3.7.4

### (a) Linear Train

We would expect the cubic regression have a lower training RSS than the linear one because if the real relationship between x and y is linear, then the explained variation for cubic regression should be higher then the linear one, so the unexplained variation(RSS) is lower for cubic regression. Also, when a statistical learning method be more flexible, the training RSS will go down.

### (b) Linear Test

We would expect the linear regression have a lower training RSS than the cubic one because if the true relationship between x and is linear, the cubic regression model will have more error and larger residual when predict y. The linear regression model would fit better and would have a lower varience.

### (c) Not Linear Train

We would expect the cubic regression have a lower training RSS than the linear one because when a statistical learning method be more flexible, the training RSS will go down.

### (d) Not Linear Testing

There not enough information to tell. Because we are not sure which model with fit data better. Both cases are possible. The one the real relationship between x and y is closer to will have a lower test RSS. 

## 3. ISLR 3.7.3 - Extra Practice 

## 3. ISLR 3.7.5 - Extra Practice 